## Import Libraries

In [67]:
import pandas as pd
import numpy as np 
from tqdm import tqdm
from PIL import Image
import os

### Create train dataset

In [68]:
df = pd.DataFrame(data= None , columns= ['label' , 'filename'])
dataset_path = os.getcwd().replace('notebook' , '') + 'output_dataset'
count = 0
for folder in [i for i in os.listdir(dataset_path) if i not in ['.DS_Store' , 'label.csv']]:
    for i , file in enumerate(os.listdir(dataset_path + '/' + folder)):
            if file.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff")):
                  df.loc[count + i] = [folder, file]
            count += len(os.listdir(dataset_path + '/' + folder))
df.to_csv(os.getcwd().replace('notebook' , '') + 'output_dataset' + '/' + 'label.csv')

## Image Processing

In [69]:
label_df_path = os.getcwd().replace('notebook' , '') + 'output_dataset' + '/' + 'label.csv' 
df = pd.read_csv(label_df_path , index_col=0)

# One Hot Encode genre file
df_encoded = pd.concat([pd.get_dummies(df, columns=['label']), df['label']], axis = 1)

In [74]:
width = 256
height = 256
x = []
for i in tqdm(range(len(df_encoded))):
    label = df_encoded['label'].iloc[i]
    filename = df_encoded['filename'].iloc[i]
    path = os.path.join(os.getcwd().replace('notebook', ''), 'output_dataset', label, filename)
    img = Image.open(path).resize((width, height))
    img = np.array(img, dtype=np.float32) / 255.0
    x.append(img)

100%|██████████| 20638/20638 [00:08<00:00, 2375.78it/s]
